In [1]:
pip install youtube_transcript_api

In [3]:
from youtube_transcript_api import YouTubeTranscriptApi

In [9]:
video_id = "p9YTmge0KsQ"
YouTubeTranscriptApi.get_transcript(video_id, languages=['pt', 'en'])

[{'duration': 5.43, 'start': 0.0, 'text': 'E aí pessoal meu nome é Douglas O'},
 {'duration': 4.74,
  'start': 3.03,
  'text': 'proprietário do canal rato borrachudo já'},
 {'duration': 4.35,
  'start': 5.43,
  'text': 'vim aqui dar um feedback oficial de tudo'},
 {'duration': 4.47,
  'start': 7.77,
  'text': 'que tá acontecendo e mais de uma semana'},
 {'duration': 6.12,
  'start': 9.78,
  'text': 'que eu postei o vídeo a Anna Sobre o'},
 {'duration': 5.97,
  'start': 12.24,
  'text': 'ocorrido o que que aconteceu comigo como'},
 {'duration': 5.37,
  'start': 15.9,
  'text': 'seria dali para frente como foi a'},
 {'duration': 4.829,
  'start': 18.21,
  'text': 'atitude do YouTube aqui no viu é o'},
 {'duration': 3.96,
  'start': 21.27,
  'text': 'primeiro vídeo Desse Canal só dá uma'},
 {'duration': 6.06,
  'start': 23.039,
  'text': 'olhada lá para você entender tudo que'},
 {'duration': 5.58,
  'start': 25.23,
  'text': 'aconteceu e depois disso eu nem o que me'},
 {'duration': 3.48